# Multi-Agent MCP Example

To start, login to azure via the CLI:

`az login --tenant <YOUR_TENANT_ID>`

`az account set -s <YOUR_SUBSCRIPTION_ID>`

In [1]:
import os, time
from dotenv import load_dotenv
from typing import Optional, Set
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import FunctionTool, ToolSet, McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval
from utils.agent_team import AgentTeam, AgentTask, _create_task
from utils.agent_trace_configurator import AgentTraceConfigurator

In [2]:
load_dotenv()
# Initialize the AI Project Client
agent_client = AgentsClient(
    endpoint=os.getenv("AI_FOUNDRY_PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential()
)

In [3]:
# Get the MCP server configuration from environment variables
mcp_server_url = os.environ.get("MCP_MSFT_DOCS_SERVER")
mcp_server_label = "msdocs"

print(f"Using MCP server: {mcp_server_label} at {mcp_server_url}")

msft_docs_mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
)

msft_docs_mcp_tool.set_approval_mode("never")

Using MCP server: msdocs at https://learn.microsoft.com/api/mcp


In [4]:
def create_task(team_name: str, recipient: str, request: str, requestor: str) -> str:
    """
    Requests another agent in the team to complete a task.

    :param team_name (str): The name of the team.
    :param recipient (str): The name of the agent that is being requested to complete the task.
    :param request (str): A description of the to complete. This can also be a question.
    :param requestor (str): The name of the agent who is requesting the task.
    :return: True if the task was successfully received, False otherwise.
    :rtype: str
    """
    task = AgentTask(recipient=recipient, task_description=request, requestor=requestor)
    team: Optional[AgentTeam] = None
    try:
        team = AgentTeam.get_team(team_name)
    except:
        pass
    if team is not None:
        team.add_task(task)
        return "True"
    return "False"


# Any additional functions that might be used by the agents:
agent_team_default_functions: Set = {
    create_task,
}

default_function_tool = FunctionTool(functions=agent_team_default_functions)

In [5]:
agent_team = AgentTeam("account_team", agents_client=agent_client)

In [6]:
account_manager_toolset = ToolSet()
account_manager_toolset.add(default_function_tool)


In [7]:
agent_team.set_team_leader(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="TeamLeader",
    instructions=(
        "You are an agent named 'TeamLeader'. You are a leader of a team of agents. The name of your team is 'account_team'. "
        "You are an agent that is responsible for receiving requests from user and utilizing a team of agents to complete the task. "
        "When you are passed a request, the only thing you will do is evaluate which team member should do which task next to complete the request. "
        "You will use the provided create_task function to create a task for the agent that is best suited for handling the task next. "
        "You will respond with the description of who you assigned the task and why. When you think that the original user request is "
        "processed completely utilizing all the talent available in the team, you do not have to create anymore tasks. "
        "Using the skills of all the team members when applicable is highly valued. "
        "Do not create parallel tasks. "
        "Here are the other agents in your team: "
        "- Specialist: You are an expert in Azure services and tools. Your task is to assist users with their Azure-related questions and tasks. Your name is Specialist. "
        "- Engineer: You are an engineer who specializes in architecting and implementing solutions on Azure. Your task is to review recommended solutions for a request and validate that they meet the requirements. Your name is Engineer. "
    ),
    toolset=account_manager_toolset,
)

In [8]:
specialist_toolset = ToolSet()
specialist_toolset.add(msft_docs_mcp_tool)

In [9]:
agent_team.add_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="Specialist",
    instructions="""
            You are an expert in Azure services and tools. Your task is to assist users with their Azure-related questions and tasks. Your name is Specialist.
            
            Use the MCP tool to: 
            - Access the latest Azure documentation and resources to provide accurate and up-to-date information.
            - Share useful links, code snippets, and explanations based on the user's request.
            - Perform tasks such as retrieving documentation, checking service status, and providing guidance on Azure best practices.

            When a user asks a question or requests assistance, you will use the MCP tool to find relevant information and provide a response. 
            If the MCP tool is not available, you will inform the user that the tool is not available and cannot assist with their request.
            Ensure that your responses are clear, concise, and relevant to the user's request. Do not provide any unverified information.
            If you are unable to assist with a request, inform the user that you cannot help with that request at this time.
    """,
    toolset=specialist_toolset,
)

In [10]:
agent_team.add_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="Engineer",
    instructions="""
            You are an engineer who specializes in architecting and implementing solutions on Azure. Your name is Engineer.
            Your task is to review recommended solutions for a request and validate that they meet the requirements.

            Use the MCP tool to: 
            - Ensure that all solutions are necessary to implement an architecture.
            - Determine if two solutions meet the same requirement and are not meant to be used in combination to implement an architecture.
            - Ensure that the selected solutions will produce an architecture that aligns with best practices.
            - Perform tasks such as retrieving documentation.            

            If two solutions meet the same requirement and are not meant to be used in combination to implement an architecture, pick the best solution, and update the recommendation. 
            Remove solutions from the recommendation that do not meet the requirements or are not necessary to implement an architecture.     
    """,
    toolset=specialist_toolset,
)

In [11]:
agent_team.assemble_team()

In [12]:
cx_request = """
    I want to build a RAG web application that uses Azure OpenAI to answer questions about my data. 
    Can you recommend an architecture for implementing this?
"""

In [13]:
agent_team.process_request(request=cx_request)

Created thread with ID: thread_19R7O6O2znMQpGxKkDQSiCJl
Starting task for agent 'TeamLeader'. Requestor: 'user'. Task description: 'Please create a task for agent in the team that is best suited to next process the following request. 
Use the _create_task function available for you to create the task. The request is: 

    I want to build a RAG web application that uses Azure OpenAI to answer questions about my data. 
    Can you recommend an architecture for implementing this?

'.
Created message with ID: msg_oo7j2dYSIDlzZ95ISL6F78yi for task in thread thread_19R7O6O2znMQpGxKkDQSiCJl
Found agent 'TeamLeader' for task recipient 'TeamLeader'
Processing task for agent 'TeamLeader'
run status for agent 'TeamLeader': RunStatus.QUEUED
Submitted tool outputs for run run_l9P8BCkTly7o2ud5E7ILxFJj in thread thread_19R7O6O2znMQpGxKkDQSiCJl
Created and processed run for agent 'TeamLeader', run ID: run_l9P8BCkTly7o2ud5E7ILxFJj
Run outputs 'TeamLeader': RunStatus.COMPLETED
Agent 'TeamLeader' comple

In [14]:
agent_team.dismantle_team()

Deleting team leader agent 'TeamLeader'
Deleting agent 'Specialist'
Deleting agent 'Engineer'
